<a href="https://colab.research.google.com/github/simran-sai/LLM-model/blob/main/LLM_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install accelerate -U


Traceback (most recent call last):
  File "<frozen importlib._bootstrap_external>", line 153, in _path_is_mode_type
  File "<frozen importlib._bootstrap_external>", line 147, in _path_stat
FileNotFoundError: [Errno 2] No such file or directory: '/usr/local/lib/python3.10/dist-packages/pip/_vendor/rich/__init__.abi3.so'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_ru

In [22]:
!pip install transformers


In [23]:
!pip install torch==1.13.1


  Using cached torch-1.13.1-cp310-cp310-manylinux1_x86_64.whl (887.5 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
xformers 0.0.20 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.


In [24]:
import pandas as pd
import torch
from transformers import TrainingArguments, Trainer, GPT2LMHeadModel, GPT2Tokenizer

In [25]:
df = pd.read_csv('Book2.csv')
df.head()

,Row No,Occasion,Relationship,Price Range,Interest,Style,Preference,Gift
0,1,Retirement,Friend,Low,Travel,Retro,Prefer unique designs,Vintage Suitcase
1,2,Farewell,Colleague,Medium,Art,Modern,Prefer practical gifts,Art Set
2,3,Valentine's Day,Partner,High,Music,Modern,Prefer convenience,Wireless Bluetooth Speaker
3,4,Job Promotion,Colleague,Low,Travel,Retro,Prefer aesthetic,Vintage Travel Journal
4,5,Retirement,Sibling,Medium,Sports,Modern,Likes aesthetic,Sports Watch


In [26]:
df.columns

Index(['Row No', 'Occasion', 'Relationship', 'Price Range', 'Interest',
       'Style', 'Preference', 'Gift'],
      dtype='object')

In [27]:
df.columns = df.columns.str.strip()


In [28]:
df['Preference']


0       Prefer unique designs
1      Prefer practical gifts
2          Prefer convenience
3            Prefer aesthetic
4             Likes aesthetic
                ...          
195        Prefer convenience
196     Prefer unique designs
197          Prefer aesthetic
198        Prefer convenience
199    Prefer practical gifts
Name: Preference, Length: 200, dtype: object

In [29]:
prompts = df.apply(lambda row: f"The occasion is {row['Occasion']}, the relationship is {row['Relationship']}, the range is {row['Price Range']},the interest is{['Interest']} ,the style is {row['Style']}, and the preference is {row['Preference']}. What is the appropriate gift?", axis=1)
answers = df['Gift']

In [30]:
prompts_and_answers = prompts + ' ' + answers
prompts_and_answers.to_csv('prompts_and_answers.txt', index=False)


In [31]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [32]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="prompts_and_answers.txt",
    block_size=128,
)


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [33]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)


In [34]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-gifts", # The output directory
    overwrite_output_dir=True, # Overwrite the content of the output directory
    num_train_epochs=3, # The number of training epochs
    per_device_train_batch_size=1, # Batch size for training
    save_steps=10_000, # After # steps model is saved
    save_total_limit=2, # The total amount of checkpoints
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)


In [38]:
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.460300


TrainOutput(global_step=603, training_loss=0.42442899675511603, metrics={'train_runtime': 1244.3507, 'train_samples_per_second': 0.485, 'train_steps_per_second': 0.485, 'total_flos': 14994692352000.0, 'train_loss': 0.42442899675511603, 'epoch': 3.0})

In [40]:
model.save_pretrained("./gpt2-gifts")
